In [2]:
import os
import pandas as pd
import pyshorteners
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [3]:
# load and concat ml excel and intern data
def list_files_in_directory(directory, file_extensions=['.csv', '.xlsx']):
    # List all files in the directory with specific extensions
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    filtered_files = [f for f in files if any(f.endswith(ext) for ext in file_extensions)]
    return filtered_files

def load_data(path_excel, path_csv, sheet_name_value=2):
    data_excel = pd.read_excel(path_excel, 
                               sheet_name=sheet_name_value, 
                               skiprows=range(1, 6))
    data_excel['STATUS'] = data_excel['STATUS'].str.strip()
    data_excel = data_excel.drop_duplicates(subset='ITEM_ID', keep='first')
    data_excel = data_excel.drop(columns=['VARIATIONS', 'VARIATION_ID'])
    
    data_csv = pd.read_csv(path_csv)

    return data_excel, data_csv
    
def select_files_and_load_data(directory):
    # List files
    files = list_files_in_directory(directory)
    if not files:
        print("No files found in the directory.")
        return None, None 
    # Display files to the user
    print("Files available:")
    for idx, file in enumerate(files):
        print(f"{idx + 1}: {file}")
    
    # Prompt the user to select files
    try:
        excel_idx = int(input("Select the Excel file (enter the number): ")) - 1
        csv_idx = int(input("Select the CSV file (enter the number): ")) - 1
        if excel_idx < 0 or excel_idx >= len(files) or csv_idx < 0 or csv_idx >= len(files):
            raise ValueError("Invalid file selection.")
    except ValueError as e:
        print(f"Error: {e}")
        return None, None
    
    # Load the data using load_data
    excel_file = os.path.join(directory, files[excel_idx])
    csv_file = os.path.join(directory, files[csv_idx])
    
    return load_data(excel_file, csv_file)
    
data_excel, data_csv = select_files_and_load_data('data')

if data_excel is not None and data_csv is not None:
    print("Excel Data:")
    print(data_excel.shape)
    print("\nCSV Data:")
    print(data_csv.shape)


def fill_sku_(df_with_sku, df_without_sku):
    """Preenche os SKUs em df_without_sku com base em df_with_sku (usando ITEM_ID como chave)."""
    merged_data = df_without_sku.merge(df_with_sku[['ITEM_ID', 'SKU']], on='ITEM_ID', how='left', suffixes=('', '_from_df1'))
    merged_data['SKU'] = merged_data['SKU_from_df1']
    merged_data.drop(columns=['SKU_from_df1'], inplace=True)
    return merged_data

df = fill_sku_(data_csv, data_excel)

print(f"\n Final df \n{df.shape}")

Files available:
1: 02-01-2025 - ANUNCIOS.csv
2: Anuncios-2025_01_06-19_07.xlsx


Select the Excel file (enter the number):  2
Select the CSV file (enter the number):  1


Excel Data:
(796, 17)

CSV Data:
(860, 21)

 Final df 
(869, 17)


In [12]:
df['CATEGORY'].unique()

array(['Relógios de bolso', 'Videogames',
       'Figuras interativas para videogames', 'Gamepads e joysticks',
       'Consoles de videogames', 'Produtos não classificados',
       'Relógios de pulso',
       'Consoles de videogames, videogames e máquinas de fliperama',
       'Figuras de ação',
       'Controles remotos multimídia para consoles de videogames',
       'Camisetas', 'Fontes de alimentação para consoles de videogames',
       'Skins para consoles e controles de videogames',
       'Relógios de parede', 'Cabos e adaptadores de áudio e vídeo',
       'Álbuns de música antigos', 'Monitores de computadores',
       'Copos e taças', 'Tênis',
       'Câmeras e barras sensoras para consoles de videogames',
       'Suportes para reprodutores de vídeo e consoles de videogames',
       'Capas e estojos para consoles de videogames',
       'Cartões de memória', 'Óculos de realidade virtual',
       'Veículos em miniatura', 'Livros', 'Camisas de futebol',
       'Álbuns de música', 

In [14]:
categorias = {
    'Videogames': '001',
    'Figuras de ação': '002',
    'Consoles de videogames': '003',
    'Gamepads e joysticks': '004',
    'Figuras interativas para videogames': '005',
    'Consoles de videogames, videogames e máquinas de fliperama': '006',
    'Relógios de pulso': '007',
    'Skins para consoles e controles de videogames': '008',
    'Camisetas': '009',
    'Livros': '010',
    'Veículos em miniatura': '011',
    'Camisas de futebol': '012',
    'Câmeras e barras sensoras para consoles de videogames': '013',
    'Controles remotos multimídia para consoles de videogames': '014',
    'Tênis': '015',
    'Brinquedos e hobbies': '016',
    'Produtos não classificados': '017',
    'Controles remotos para reprodutores de vídeo': '018',
    'Óculos de realidade virtual': '019',
    'Relógios de parede': '020',
    'Relógios de bolso': '021',
    'Luminárias de mesa e escritório': '022',
    'Canecas': '023',
    'Suportes para reprodutores de vídeo e consoles de videogames': '024',
    'Copos e taças': '025',
    'Álbuns de música antigos': '026',
    'Capas e estojos para consoles de videogames': '027',
    'Cartões de memória': '028',
    'Fontes de alimentação para consoles de videogames': '029',
    'Cabos e adaptadores de áudio e vídeo' : '030'
}
df['CATEGORY_ID'] = df['CATEGORY'].map(categorias)
df.columns


Index(['ITEM_ID', 'SKU', 'TITLE', 'QUANTITY', 'CHANNEL', 'MARKETPLACE_PRICE',
       'MSHOPS_PRICE', 'MSHOPS_PRICE_SYNC', 'CURRENCY_ID', 'DESCRIPTION',
       'SHIPPING_METHOD_MARKETPLACE', 'SHIPPING_METHOD_MSHOPS', 'LISTING_TYPE',
       'FEE_PER_SALE_MARKETPLACE', 'FEE_PER_SALE_MSHOPS', 'STATUS', 'CATEGORY',
       'CATEGORY_ID'],
      dtype='object')

In [16]:
def update_product_skus(data):
    
    current_year_month = datetime.now().strftime("%y%m")  # Ano e mês atual no formato "yyMM"
    
    # Preparando para rastrear informações adicionais sobre novos SKUs
    new_skus_list = []
    new_skus_details = []  # Lista para armazenar detalhes para novos SKUs incluindo ITEM_ID e TITLE

    # Certifique-se de que CATEGORY_ID tenha 3 dígitos
    data['CATEGORY_ID'] = data['CATEGORY_ID'].str.zfill(3)

    # Dicionário para rastrear o contador por categoria e mês
    category_counters = {}

    for category_id in data['CATEGORY_ID'].unique():
        category_mask = data['CATEGORY_ID'] == category_id

        # Filtra os SKUs existentes dessa categoria e mês
        existing_skus = data.loc[category_mask & data['SKU'].notna(), 'SKU']

        # Verifica o ano/mês atual para a categoria
        year_month = current_year_month

        # Extraímos os contadores do SKU, considerando a categoria e mês
        counters = existing_skus.str.extract(f"^{category_id}-{year_month}-(\\d{{4}})$")[0]
        valid_counters = pd.to_numeric(counters, errors='coerce').dropna().astype(int)
        
        # Define o próximo contador, baseado no maior contador existente, ou inicia em 1
        if not valid_counters.empty:
            next_counter = valid_counters.max() + 1
        else:
            next_counter = 1

        # Filtra os índices dos produtos sem SKU
        null_skus_indices = data.index[category_mask & data['SKU'].isna()]
        for idx in null_skus_indices:
            # Gera um SKU único com base na categoria, ano/mês e o contador
            new_sku = f"{category_id}-{year_month}-{next_counter:04d}"
            data.at[idx, 'SKU'] = new_sku
            new_skus_list.append(new_sku)
            new_skus_details.append({
                'TITLE': data.at[idx, 'TITLE'],
                'SKU': new_sku,
                'ITEM_ID': data.at[idx, 'ITEM_ID'],
            })

            # Incrementa o contador após gerar o SKU para o próximo
            next_counter += 1

    # Converte a lista de dicionários em DataFrame
    new_skus_data = pd.DataFrame(new_skus_details)  # Agora inclui ITEM_ID e TITLE

    return data, new_skus_data

def get_links(data):
    data['ITEM_LINK'] = data['ITEM_ID'].apply(
        lambda x: f"https://www.mercadolivre.com.br/anuncios/lista?filters=OMNI_ACTIVE|OMNI_INACTIVE|CHANNEL_NO_PROXIMITY_AND_NO_MP_MERCHANTS&page=1&search={x[3:]}" if pd.notnull(x) else "")

    data['URL'] = data.apply(
        lambda row: f"https://www.collectorsguardian.com.br/{row['ITEM_ID'][:3]}-{row['ITEM_ID'][3:]}-{row['TITLE'].replace(' ', '-').lower()}-_JM#item_id={row['ITEM_ID']}", 
        axis=1)

    return data
    
data, news = update_product_skus(df)
print(news)
data = get_links(data)
data.head(2)


                                                TITLE            SKU  \
0   Metal Gear Rising: Revengeance - Special Editi...  001-2501-0005   
1                       Biohazard Revelations 2 - Ps3  001-2501-0006   
2                Biohazard Revival Selection - Hd Ps3  001-2501-0007   
3      Amiibo Donkey Kong - Lacrado Nintendo Original  005-2501-0001   
4        Sony Playstation Dualshock 3 - Preto (usado)  004-2501-0002   
5   Playstation 4 20th Anniversary Limited Edition...  003-2501-0001   
6       Memory Card De 8mb Para O Playstation 2 (ps2)  006-2501-0001   
7   Vmu (visual Memory Unit) Verde - Sega Dreamcas...  006-2501-0002   
8   Vmu (visual Memory Unit) - Preto -  Sega Dream...  006-2501-0003   
9   Vmu (visual Memory Unit) Vermelho- Sega Dreamc...  006-2501-0004   
10         Figure Pikachu E Eevee Original/licenciado  002-2501-0003   
11            Ocelot Bust Figure - Metal Gear Solid V  002-2501-0004   
12     Suporte Vertical Ps4 Sony - Usado (incompleto)  024-2501-

,ITEM_ID,SKU,TITLE,QUANTITY,CHANNEL,MARKETPLACE_PRICE,MSHOPS_PRICE,MSHOPS_PRICE_SYNC,CURRENCY_ID,DESCRIPTION,SHIPPING_METHOD_MARKETPLACE,SHIPPING_METHOD_MSHOPS,LISTING_TYPE,FEE_PER_SALE_MARKETPLACE,FEE_PER_SALE_MSHOPS,STATUS,CATEGORY,CATEGORY_ID,ITEM_LINK,URL
0,MLB3420123395,021-2410-0001,Pocket Watch Limited Edition Super Mário Bros ...,1,Mercado Livre e Mercado Shops,400.0,350.0,No Vincular,R$,Apresentando o Relógio de Bolso Super Mario Br...,Mercado Envios grátis,Mercado Envios grátis,Premium,NaN,NaN,Ativo,Relógios de bolso,021,https://www.mercadolivre.com.br/anuncios/lista...,https://www.collectorsguardian.com.br/MLB-3420...
1,MLB3607686939,001-2410-0462,Rain - Ps3 Exclusivo Sony - Jogo Incomum - Jap...,1,Mercado Livre e Mercado Shops,600.0,450.0,No Vincular,R$,Este título cativante é cobiçado por coleciona...,Envios por conta própria,Mercado Envios por conta do comprador,Premium,NaN,NaN,Ativo,Videogames,001,https://www.mercadolivre.com.br/anuncios/lista...,https://www.collectorsguardian.com.br/MLB-3607...


In [18]:
data.shape

(869, 20)

In [20]:
data.isna().sum(
    
)

ITEM_ID                          0
SKU                              0
TITLE                            0
QUANTITY                         0
CHANNEL                          0
MARKETPLACE_PRICE                0
MSHOPS_PRICE                     0
MSHOPS_PRICE_SYNC                0
CURRENCY_ID                      0
DESCRIPTION                     17
SHIPPING_METHOD_MARKETPLACE      0
SHIPPING_METHOD_MSHOPS           0
LISTING_TYPE                     0
FEE_PER_SALE_MARKETPLACE       869
FEE_PER_SALE_MSHOPS            869
STATUS                           0
CATEGORY                         0
CATEGORY_ID                      2
ITEM_LINK                        0
URL                              0
dtype: int64

In [22]:

def extract_first_src(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
    
        img_tag = soup.find('img', class_='ui-pdp-image ui-pdp-gallery__figure__image')
        if img_tag and 'src' in img_tag.attrs:
            return img_tag['src']
        return None  # Caso a tag ou atributo não seja encontrado
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return None

# Aplicando a função a cada link no DataFrame
df['IMG'] = df['URL'].apply(extract_first_src)

# Exibindo o DataFrame com a nova coluna
df.shape


Erro ao processar https://www.collectorsguardian.com.br/MLB-3783999901-assassins-creed---playstation-3-_JM#item_id=MLB3783999901: 429 Client Error: Too Many Requests for url: https://www.collectorsguardian.com.br/MLB-3783999901-assassins-creed---playstation-3-_JM#item_id=MLB3783999901
Erro ao processar https://www.collectorsguardian.com.br/MLB-3869330843-yoshi-island-super-nintendo-original-_JM#item_id=MLB3869330843: 429 Client Error: Too Many Requests for url: https://www.collectorsguardian.com.br/MLB-3869330843-yoshi-island-super-nintendo-original-_JM#item_id=MLB3869330843
Erro ao processar https://www.collectorsguardian.com.br/MLB-3936838915-capa-covers-ps5-final-fantasy-xvi-versão-com-disco-_JM#item_id=MLB3936838915: 429 Client Error: Too Many Requests for url: https://www.collectorsguardian.com.br/MLB-3936838915-capa-covers-ps5-final-fantasy-xvi-vers%C3%A3o-com-disco-_JM#item_id=MLB3936838915


(869, 21)

In [19]:
df.columns

Index(['ITEM_ID', 'SKU', 'TITLE', 'QUANTITY', 'CHANNEL', 'MARKETPLACE_PRICE',
       'MSHOPS_PRICE', 'MSHOPS_PRICE_SYNC', 'CURRENCY_ID', 'DESCRIPTION',
       'SHIPPING_METHOD_MARKETPLACE', 'SHIPPING_METHOD_MSHOPS', 'LISTING_TYPE',
       'FEE_PER_SALE_MARKETPLACE', 'FEE_PER_SALE_MSHOPS', 'STATUS', 'CATEGORY',
       'CATEGORY_ID', 'ITEM_LINK', 'URL', 'IMG'],
      dtype='object')

In [24]:
# Exportar múltiplos dataframes para um único arquivo Excel
with pd.ExcelWriter("06-01-2025.xlsx", engine='openpyxl') as writer:
    data.to_excel(writer, sheet_name='PRODUTOS', index=False)

